# Using Forks

Forks provide the ability to grab on-chain state and pull it into a local EVM. In this example, we'll show how to:
- Pull information/state from the live USDC contract
- Dump state to a file
- Load the state into a local EVM
- Pretend to be a given user and interact with the USDC contract

You can learn more about the methods available on the USDC contract here: [Etherscan](https://etherscan.io/address/0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48#readProxyContract)

See the `dump_usdc_state.py` script for an example of pulling and saving the state information. 


## Setup
Make the required imports

In [15]:
from simular import (
    PyEvm,
    contract_from_inline_abi,
    create_account,
)

Set the addresses we need to use

In [16]:
# USDC Contract address
USDC = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
# Address of the real master minter from the on-chain contract
MM = "0xe982615d461dd5cd06575bbea87624fda4e3de17"

import json
with open('./usdc_addresses.json') as f:
    minters = json.loads(f.read())

# Minters created in the dump script
M1 = minters['m1']
M2 = minters['m2']
M3 = minters['m3']

Load the EVM state from file.  See `usdc_cache.json` in this directory.  
You'll notice it includes the address above and the bytecode for 2 contracts used by USDC.

In [17]:
with open('./usdc_cache.json') as f:
    cache = f.read()

Now create an EVM and load the state

In [18]:
evm = PyEvm.from_snapshot(cache)

To call USDC, we need a contract. We'll specify the methods we want to use.  They are
part of the USDC contract.

In [19]:
contract_methods = [
"function isMinter(address) (bool)",
"function totalSupply() (uint256)",
"function mint(address, uint256) (bool)",
"function burn(uint256)",
"function minterAllowance(address) (uint256)"]

usdc = contract_from_inline_abi(evm, contract_methods)

We need to point `usdc` to the correct contract, which is the USDC contract

In [20]:
usdc.at(USDC)

If you look at the `dump_usdc_state.py` script, you'll 
see we allocated the ability for each minter to mint 10000000. Let's check it...

In [21]:
assert 10000000 == usdc.minterAllowance.call(M1)

Let's verify the no one has minted any USDC yet...

In [22]:
assert 0 == usdc.totalSupply.call()

Let's mint some!

In [23]:
# each mints some to themselves
assert usdc.mint.transact(M1, 10000000, caller=M1)
assert usdc.mint.transact(M2, 10000000, caller=M2)
assert usdc.mint.transact(M3, 10000000, caller=M3)

In [24]:
assert 30000000 == usdc.totalSupply.call()

Just for the heck of it, let's see if bob can mint some...

In [25]:
bob = create_account(evm)
try: 
    usdc.mint.transact(bob, 50000000, caller=bob)
except: 
    print("NOPE! Not allowed bob!")

NOPE! Not allowed bob!


In [26]:
assert 30000000 == usdc.totalSupply.call()

Now, lets' trying burning some...

In [27]:
usdc.burn.transact(10000000, caller=M2)

[]

In [28]:
assert 20000000 == usdc.totalSupply.call()

And that's it.  Here we showed pulling contract code and state from the on-chain USDC contract. 